# Nuova sezione

In [ ]:
import pandas as pd

df = pd.read_csv('games_final.csv')
tm = pd.read_csv('teams.csv')
#df = df.dropna()

In [ ]:
df.sort_values(by = 'GAME_DATE_EST', inplace = True)
df.reset_index(inplace=True, drop = True)
df


ASSEGNA ID AL NOME DELLA SQUADRA

In [ ]:
id=tm['TEAM_ID']
name=tm['ABBREVIATION']
dict={}
for i in range(0,30):
  dict[id[i]]=name[i]
df["HOME_TEAM"] = df["HOME_TEAM_ID"].map(dict)

In [ ]:
df["VISITOR_TEAM"] = df["VISITOR_TEAM_ID"].map(dict)

Creazione nuove colonne

In [ ]:
df.insert(4,'HOME_TEAM','',True)
df.insert(5,'H_ELO_BEFORE','',True)
df.insert(6,'H_ELO_AFTER','',True)
df.insert(7,'Win_Streak_H','',True)
df.insert(8,'Win_last_10_H','',True)
df.insert(9,'Avg_points_last_10_H','',True)
df.insert(11,'VISITOR_TEAM','',True)
df.insert(12,'V_ELO_BEFORE','',True)
df.insert(13,'V_ELO_AFTER','',True)
df.insert(14,'Win_Streak_V','',True)
df.insert(15,'Win_last_10_V','',True)
df.insert(16,'Avg_points_last_10_V','',True)

Calcolo ELO

In [ ]:
import math
def win_probs(home_elo, away_elo, home_court_advantage) :
  h = math.pow(10, home_elo/400)
  r = math.pow(10, away_elo/400)
  a = math.pow(10, home_court_advantage/400) 

  denom = r + a*h
  home_prob = a*h / denom
  away_prob = r / denom 
  
  return home_prob, away_prob
def elo_k(MOV, elo_diff):
  k = 20
  if MOV>0:
      multiplier=(MOV+3)**(0.8)/(7.5+0.006*(elo_diff))
  else:
      multiplier=(-MOV+3)**(0.8)/(7.5+0.006*(-elo_diff))
  return k*multiplier

def update_elo(home_score, away_score, home_elo, away_elo,home_court_advantage) :
  home_prob, away_prob = win_probs(home_elo, away_elo, home_court_advantage) 

  if (home_score - away_score > 0) :
    home_win = 1 
    away_win = 0 
  else :
    home_win = 0 
    away_win = 1 
  
  k = elo_k(home_score - away_score, home_elo - away_elo)
  #k=20
  updated_home_elo = home_elo + k * (home_win - home_prob) 
  updated_away_elo = away_elo + k * (away_win - away_prob)

  return updated_home_elo, updated_away_elo



Inseriamo ELO nel dataframe

In [ ]:
def get_prev_elo(team, date, season) :
  prev_game = df.loc[(df['GAME_DATE_EST'] < date) & ((df['HOME_TEAM_ID'] == team) | (df['VISITOR_TEAM_ID'] == team))].sort_values(by = 'GAME_DATE_EST').tail(1).iloc[0]
  if team == prev_game['HOME_TEAM_ID'] :
    elo_rating = prev_game['H_ELO_AFTER']
  else :
    elo_rating = prev_game['V_ELO_AFTER']
  if prev_game['SEASON'] != season :
    return (0.75 * elo_rating) + (0.25 * 1505)
  else :
    return elo_rating


df.sort_values(by = 'GAME_DATE_EST', inplace = True)
df.reset_index(inplace=True, drop = True)
teams=[]

for index, row in df.iterrows():
  game_id = row['GAME_ID']
  game_date = row['GAME_DATE_EST']
  season = int(row['SEASON'])
  h_team, a_team = int(row['HOME_TEAM_ID']), int(row['VISITOR_TEAM_ID'])
  h_score, a_score = int(row['PTS_home']), int(row['PTS_away'])
  if (h_team not in teams) :
    h_elo_before = 1500
    teams.append(h_team)
  else :
    h_elo_before = get_prev_elo(h_team, game_date, season)

  if (a_team not in teams) :
    a_elo_before = 1500
    teams.append(a_team)
  else :
    a_elo_before = get_prev_elo(a_team, game_date, season)

  h_elo_after, a_elo_after = update_elo(h_score, a_score, h_elo_before, a_elo_before, 100)
  df.iloc[index,5]=h_elo_before
  df.iloc[index,6]=h_elo_after
  df.iloc[index,12]=a_elo_before
  df.iloc[index,13]=a_elo_after
  print(index)

inserisco i dati delle ultime 10 partite

In [ ]:
def get_prev_games(team, date) :
  wins_streak=0
  wins=0
  mean_points=0
  prev_games = df.loc[(df['GAME_DATE_EST'] < date) & ((df['HOME_TEAM_ID'] == team) | (df['VISITOR_TEAM_ID'] == team))].sort_values(by = 'GAME_DATE_EST').tail(10)
  prev_games.sort_values(by = 'GAME_DATE_EST',inplace=True)
  for index,row in prev_games.iterrows():
    if team == row['HOME_TEAM_ID']:
      mean_points= mean_points + row['PTS_home']
      if row['HOME_TEAM_WINS'] == 1:
        wins_streak=wins_streak + 1
        wins=wins+1
      else:
        wins_streak=0
    else:
      mean_points= mean_points + row['PTS_away']
      if row['HOME_TEAM_WINS'] == 1:
        wins_streak=0
      else:
        wins_streak=wins_streak+1
        wins=wins+1
  if mean_points != 0:
     mean_points= mean_points / len(prev_games)
  return wins_streak,wins,mean_points



for index, row in df.iterrows():
  game_date = row['GAME_DATE_EST']
  h_team, a_team = int(row['HOME_TEAM_ID']), int(row['VISITOR_TEAM_ID'])
  
  win_s,win,avg_p = get_prev_games(h_team,game_date)
  df.iloc[index,7]=win_s
  df.iloc[index,8]=win
  df.iloc[index,9]=avg_p
  win_s,win,avg_p = get_prev_games(a_team,game_date)
  df.iloc[index,14]=win_s
  df.iloc[index,15]=win
  df.iloc[index,16]=avg_p
  print(index)




Prediction

In [ ]:
#cols = [1,2,3,5,6,7]
#df.drop(df.columns[cols],axis=1, inplace=True)

In [ ]:
df=df.drop('Unnamed: 0', axis=1)
cols = [0,1,2,3,4,6,10,11,13,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31]
df.drop(df.columns[cols],axis=1, inplace=True)

In [ ]:
features = df.drop(columns = 'HOME_TEAM_WINS')
label = df['HOME_TEAM_WINS']

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features, label, test_size = 0.2, shuffle = True)

In [ ]:
features

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

rf = RandomForestClassifier(random_state=42, max_features='sqrt')
rf.fit(x_train, y_train)
y_pred_rf = rf.predict(x_test)
print(metrics.accuracy_score(y_test, y_pred_rf))

from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

In [ ]:
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import KFold,train_test_split,cross_val_score

#Create a svm Classifier
clf = svm.SVC(kernel='rbf') # gaussian Kernel

kfold = KFold(n_splits=10, random_state=1,shuffle=True)

results = cross_val_score(clf, features, label, cv=kfold,scoring='accuracy')

#Train the model using the training sets
#clf.fit(x_train, y_train)

#Predict the response for test dataset
#y_pred = clf.predict(x_test)

#print("Accuracy:",metrics.accuracy_score(y_test, y_pred))



In [ ]:
results

In [ ]:
import statistics 
media=statistics.mean(results)
dev_st=statistics.stdev(results)
print(media,dev_st)

In [ ]:
import statistics 
media=statistics.mean(results)
dev_st=statistics.stdev(results)
print(media,dev_st)

In [ ]:
gammas = [0.1,0.2,0.4,0.6,0.8,1,1.2,1.4,1.6,1.8,2]
for gamma in gammas:
  svc = svm.SVC(kernel='rbf', gamma=gamma).fit(x_train, y_train)
  y_pred = svc.predict(x_test)
  print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


In [ ]:
cs = [1,1.4]
for c in cs:
  svc = svm.SVC(kernel='rbf', C=c).fit(x_train, y_train)
  y_pred = svc.predict(x_test)
  print("Accuracy:",metrics.accuracy_score(y_test, y_pred))